In [2]:
from sympy import init_session
import numpy as np
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial
init_session()
from tqdm import tqdm

IPython console for SymPy 1.6.2 (Python 3.7.3-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.6.2/



In [1]:
from sympy import *

In [3]:
from itertools import product
# X = Poly(x ** 5 - x ** 4 - 2 * x ** 3 - 8 * x ** 2 + 6 * x - 1)



In [4]:
def get_factor(number: int):
    factors = list(factorint(number).keys())
    factors.append(number)
    factors.append(1)
    factors.append(-1)
    factors.extend([-x for x in factors.copy()])
    return np.unique(np.array(factors))

def cartesian_product(*arrays):
    la = len(arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=np.int64)
    for i, a in tqdm(enumerate(np.ix_(*arrays))):
        arr[...,i] = a
    return arr.reshape(-1, la)

In [5]:
x, y, z, t = symbols('x y z t')
# исходный полином
X = Poly(x ** 5 - x ** 4 - 2 * x ** 3 - 8 * x ** 2 + 6 * x - 1)
# X = Poly(random_poly(x, 10, 25, 100))
print(X)
# numpy_poly_coef = np.poly1d(X.all_coeffs())# input array
# print(numpy_poly_coef)
n = degree(X) // 2
R = np.array(range(0,n+1))
factors = list()
# раскладываем множество делителей X (i), где i эт
for i in tqdm(range(0, n+1)):
    t = X(i)
    factors.append(get_factor(t))
    factors[i].sort()
factors_len = 1
for f in factors:
    factors_len = f.shape[0] * factors_len
factors_len = factors_len // 2
# print(factors)
# print(len(factors))
print(factors_len)
ys = product(*factors)
for i, y_i in tqdm(enumerate(ys), total=factors_len):
    # print("-------------------------")
    y_i = np.array(y_i)
    # print(y_i)
    # print(R)
    k = lagrange(R, y_i)
    # print(k)
    buf_poly = Poly(k.c, x)
    # print(buf_poly_coef)
    if degree(buf_poly) > 0:
        quo, rem =  X.div(buf_poly)
        # print(rem)
        if rem == 0:
            print(y_i)
            print(buf_poly)
            print(quo)
            print(quo * buf_poly)
            print("-------------------------")
    if i >= factors_len:
        break

 14%|█▍        | 13949/98304 [01:51<11:16, 124.62it/s]


Poly(32*x**10 + 55*x**9 + 86*x**8 + 83*x**7 + 52*x**6 + 37*x**5 + 92*x**4 + 32*x**3 + 96*x**2 + 58*x + 76, x, domain='ZZ')
98304


KeyboardInterrupt: 

In [6]:
def brute_force(y_i):
    global R
    # print(R)
    k = lagrange(R, y_i)
    buf_poly = Poly(k.c, x)
    # print(buf_poly_coef)
    if degree(buf_poly) > 0:
        quo, rem =  X.div(buf_poly)
        # print(rem)
        if rem == 0:
            return quo, y_i
        else:
            return None
    else:
        return None

In [ ]:
import concurrent.futures
import time
x, y, z, t = symbols('x y z t')
# X = Poly(x ** 5 - x ** 4 - 2 * x ** 3 - 8 * x ** 2 + 6 * x - 1)
X = Poly(random_poly(x, 10, 25, 100))
print(X)
# numpy_poly_coef = np.poly1d(X.all_coeffs())# input array
# print(numpy_poly_coef)
n = degree(X) // 2
R = np.array(range(0,n+1))
factors = list()
for i in tqdm(range(0, n+1)):
    t = X(i)
    factors.append(get_factor(t))
    factors[i].sort()
factors_len = 1
for f in factors:
    factors_len = f.shape[0] * factors_len
factors_len = factors_len // 2
print(factors_len)
ys = product(*factors)
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for res in tqdm(executor.map(brute_force, ys)):
        if res is not None:
            print(res)

# for i, y_i in tqdm(enumerate(ys), total=factors_len):
#     # print("-------------------------")
#     y_i = np.array(y_i)
#     # print(y_i)
#     # print(R)
#     k = lagrange(R, y_i)
#     # print(k)
#     buf_poly = Poly(k.c, x)
#     # print(buf_poly_coef)
#     if degree(buf_poly) > 0:
#         quo, rem =  X.div(buf_poly)
#         # print(rem)
#         if rem == 0:
#             print(y_i)
#             print(buf_poly)
#             print(quo)
#             print(quo * buf_poly)
#             print("-------------------------")
#     if i >= factors_len:
#         break

100%|██████████| 6/6 [00:00<00:00, 2998.07it/s]
3084it [01:15, 40.64it/s]  


Poly(92*x**10 + 47*x**9 + 26*x**8 + 60*x**7 + 45*x**6 + 100*x**5 + 52*x**4 + 57*x**3 + 80*x**2 + 45*x + 88, x, domain='ZZ')
163840


In [17]:
X = Poly(random_poly(x, 10, 25, 100))
print(X)
t = factor_list(X)
print(t)

Poly(61*x**10 + 71*x**9 + 78*x**8 + 50*x**7 + 85*x**6 + 40*x**5 + 32*x**4 + 89*x**3 + 26*x**2 + 52*x + 25, x, domain='ZZ')
(1, [(Poly(61*x**10 + 71*x**9 + 78*x**8 + 50*x**7 + 85*x**6 + 40*x**5 + 32*x**4 + 89*x**3 + 26*x**2 + 52*x + 25, x, domain='ZZ'), 1)])


In [13]:
len("116837345136530503245738768505655316569055743")

In [13]:
mul_ = 1

print(mul_)

338688000000


In [15]:
Poly([1,5,8], x)